In [ ]:
# !pip install pandas-datareader
!pip install "yfinance[nospam]"

In [ ]:
import requests
import bs4 as bs
import pickle
import os
import datetime as dt
import pandas as pd
import numpy as np
import pandas_datareader as web
import time
from lxml import html
import os
import yfinance as yf
from datetime import datetime

In [ ]:
def save_sp500_tickers():
    # Mengambil halaman dan membuat soup
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, "lxml")
    
    # Mencari tabel yang berisi data ticker
    table = soup.find('table', {'id': 'constituents'})
    
    tickers = []
    # Mengiterasi setiap baris tabel kecuali header
    for row in table.findAll('tr')[1:]:
        # Mendapatkan data dari tiga kolom yang diinginkan
        ticker = row.find_all('td')[0].text.strip()
        company_name = row.find_all('td')[1].text.strip()
        sector = row.find_all('td')[3].text.strip()
        
        # Menyimpan data ke list sebagai tuple
        tickers.append({'ticker' : ticker, 'company name' : company_name, 'sektor' : sector})

    # Menyimpan data ticker ke file pickle
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)

    # Mencetak daftar ticker untuk verifikasi
    print(tickers)

    return tickers

# Memanggil fungsi untuk menjalankan dan menyimpan ticker
save_sp500_tickers()

In [ ]:
# Get Timeseries data in Yahoo
def get_data_from_yahoo(reload_sp500 = False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs2'):
        os.makedirs('stock_dfs2')
    
    start = dt.datetime(2000,1,1)
    end = dt.datetime(2020,5,28)

    # Grab all ticker data
    for s in tickers[401:500]:
        print(s)
        stock = s['ticker']
        if not os.path.exists('stock_dfs2/{}.csv'.format(stock)):
            try:
                kk = yf.Ticker(stock)
                end_date = datetime.now().strftime('%Y-%m-%d')
                df = kk.history(start='2020-01-01',end=end_date,interval='1d')
                df.to_csv('stock_dfs2/{}.csv'.format(stock))
            except:
                print(f"Problems retrieving data for {stock}. Skipping!")
            else:
                print('Already have {}'.format(stock))

get_data_from_yahoo()



In [2]:
def scrape_data_yahoo(stock_code,endpoint):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1', 
        'Connection': 'keep-alive'
    }

    url = 'https://finance.yahoo.com/quote/' + stock_code + '/' + endpoint + '/'
    response = requests.get(url, headers=headers)
    print(response)
    tree = html.fromstring(response.content)
    divs = tree.xpath("//div[contains(@class, 'column') and contains(@class, 'yf-1ezv2n5')]")
    header = []
    for div in divs:
        text = div.text_content().strip() 
        header.append(text)
    feature=['Date']
    body = tree.xpath("//div[contains(@class, 'column') and contains(@class, 'sticky') and contains(@class, 'yf-1xjz32c')]")
    for isi in body:
        text = isi.text_content().strip() 
        feature.append(text)
    isi_data =[]
    data = tree.xpath("//div[contains(@class, 'column') and not(contains(@class, 'sticky')) and contains(@class, 'yf-1xjz32c')]")
    for isi in data:
        text = isi.text_content().strip() 
        isi_data.append(text)
    print(isi_data)
    list_kelompok=[[header[i] for i in range(1,len(header))]]
    jumlah_per_kelompok = len(header)-1
    for i in range(0, len(isi_data), jumlah_per_kelompok):
        kelompok = isi_data[i:i + jumlah_per_kelompok]
        list_kelompok.append(kelompok)
    df = pd.DataFrame(columns=feature)
    n = np.array(list_kelompok).T.tolist()
    df = pd.DataFrame(n,columns=feature)
    return df

def scrape_lapkeu(stock_code):
    if not os.path.exists('Laporan_keuangan'):
        os.makedirs('Laporan_keuangan')
    df = scrape_data_yahoo(stock_code,'financials')
    df2 = scrape_data_yahoo(stock_code,'balance-sheet')
    df3 = scrape_data_yahoo(stock_code,'cash-flow')
    df_gabungan  = pd.merge(df, df2, on='Date', how='outer')
    df_gabungan  = pd.merge(df_gabungan, df3, on='Date', how='outer')
    df_gabungan
    df_gabungan.to_csv('Laporan_keuangan/'+stock_code+'_laporan_keuangan.csv', index=False)

# scrape_lapkeu('AVGO')

In [69]:
def get_lapkeu(reload_sp500 = False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    for i,s in enumerate(tickers[498:499]):
        stock = s['ticker']
        if '.' in stock:
            stock = stock.replace('.','-')
        print(i, stock)
        scrape_lapkeu(stock)
        time.sleep(2)

# def lengkapi_data_lapkeu():
#     dir = './Laporan_keuangan'
#     for i in os.listdir(dir)[:]:
#         file = os.path.join(dir, i)
#         # print(file)
#         if not os.path.isfile(file):
#             # print("masuk 1")
#             continue
#         if  os.path.getsize(file) > 1000:
#             # print("masuk 2")
#             continue
#         kode = i.split('_')[0]
#         kode = kode.replace('.','-')
#         print(kode)
#         scrape_lapkeu(kode)
#         c


In [70]:
get_lapkeu()

0 XYL
<Response [200]>
['8,396,000.00', '7,364,000.00', '5,522,000.00', '5,195,000.00', '4,876,000.00', '--', '5,305,000.00', '4,647,000.00', '3,438,000.00', '3,220,000.00', '3,046,000.00', '--', '3,091,000.00', '2,717,000.00', '2,084,000.00', '1,975,000.00', '1,830,000.00', '--', '2,154,000.00', '1,989,000.00', '1,573,000.00', '1,383,000.00', '1,330,000.00', '--', '937,000.00', '728,000.00', '511,000.00', '592,000.00', '500,000.00', '--', '-25,000.00', '-21,000.00', '-34,000.00', '-69,000.00', '-70,000.00', '--', '-74,000.00', '-72,000.00', '-37,000.00', '-12,000.00', '-145,000.00', '--', '838,000.00', '635,000.00', '440,000.00', '511,000.00', '285,000.00', '--', '73,000.00', '26,000.00', '85,000.00', '84,000.00', '31,000.00', '--', '765,000.00', '609,000.00', '355,000.00', '427,000.00', '254,000.00', '--', '765,000.00', '609,000.00', '355,000.00', '427,000.00', '254,000.00', '--', '3.25', '2.81', '1.97', '2.37', '1.41', '--', '3.24', '2.79', '1.96', '2.35', '1.40', '--', '241,600.00'

In [67]:
lengkapi_data_lapkeu()

In [15]:
os.listdir('./Laporan_keuangan')[:10]

['AAL_laporan_keuangan.csv',
 'AAPL_laporan_keuangan.csv',
 'ABBV_laporan_keuangan.csv',
 'ABNB_laporan_keuangan.csv',
 'ABT_laporan_keuangan.csv',
 'ACGL_laporan_keuangan.csv',
 'ACN_laporan_keuangan.csv',
 'ADBE_laporan_keuangan.csv',
 'ADI_laporan_keuangan.csv',
 'ADM_laporan_keuangan.csv']